In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta
from pathlib import Path

# Prepare datasets for DDAS

## 0. User inputs

In [2]:
# Define input files 
# 1. Raw water chemistry data from rivers
wc_raw_data_dir = Path("../../data/river/water_chemistry/raw")
wc_raw_data_file_paths = list(wc_raw_data_dir.glob("water_chem_*_raw_18-06-2025.csv"))

# 2. Stations metadata
stn_metadata_file_paths = list(wc_raw_data_dir.glob("water_chem_*_metadata_18-06-2025.csv"))

# 3. Parameters metadata
pars_metadata_file_path = "../../data/river/water_chemistry/raw/water_chemistry_parameters_units.csv"

# Define a mapping from parameter names to long names
standard_name_map = {
    "DOC": "Dissolved Organic Carbon",
    "Color": "Water Color",
    "NH4-N": "Ammonium-N",
    "NO3-N": "Nitrate-N",
    "TRP": "Total Reactive Phosphorus",
    "POC": "Particulate Organic Carbon",
    "SPM": "Suspended Particulate Matter", 
    "Si": "Silicon",
    "SiO2": "Silicate", 
    "TOC": "Total Organic Carbon",
    "TOTN": "Total Nitrogen",
    "TOTN_EF_usikker": "Total Nitrogen (Eurofins)",
    "TOTP": "Total Phosphorus",
    "TDP": "Total Dissolved Phosphorus",
    "PP": "Particulate Phosphorus",
    "TSM": "Total Suspended Matter",
    "UV_Abs_254nm": "Ultraviolet Absorbance 254nm",
    "UV_Abs_410nm": "Ultraviolet Absorbance 410nm",
}

# Where to save 
output_dir = Path("../../data/river/water_chemistry/raw") 
output_dir.mkdir(parents=True, exist_ok=True)

## 1. Load data

In [3]:
# Read raw water chemistry data and metadata
wc_raw_dfs = [pd.read_csv(fp) for fp in wc_raw_data_file_paths]
stn_metadata_dfs = [pd.read_csv(fp) for fp in stn_metadata_file_paths]
pars_metadata_df = pd.read_csv(pars_metadata_file_path)

## 2. Create datasets and assign metadata

In [4]:
rivers_raw_datasets = []

for df, meta_df in zip(wc_raw_dfs, stn_metadata_dfs):
    # Make sure date is datetime
    df["sample_date"] = pd.to_datetime(df["sample_date"])

    # Remove constant fields from dataframe to avoid them being treated as varying
    constant_fields = ["station_id", "station_code", "station_name", "station_type"]
    df_clean = df.drop(columns=constant_fields, errors="ignore")

    # Convert dataframe to dataset
    ds = xr.Dataset.from_dataframe(df_clean.set_index("sample_date"))

    # Add attributes to sample date (time coordinate)
    ds["sample_date"].attrs.update({"standard_name": "Time", "long_name": "Time of measurement", "axis": "T"})

    # Add coordinates 
    lat = meta_df["latitude"].iloc[0]
    lon = meta_df["longitude"].iloc[0]
    ds = ds.assign_coords(
        latitude=xr.DataArray(lat, dims=(), attrs={"standard_name": "latitude", "long_name": "latitude","units": "degree_north"}),
        longitude=xr.DataArray(lon, dims=(), attrs={"standard_name": "longitude", "long_name": "longitude", "units": "degree_east"})
    )

    # Add station info
    meta_row = meta_df.iloc[0]
    ds["station_id"] = xr.DataArray(meta_row["station_id"], dims=())
    ds["station_code"] = xr.DataArray(meta_row["station_code"], dims=())
    ds["station_name"] = xr.DataArray(meta_row["station_name"], dims=(), attrs={"cf_role": "timeseries_id"})
    ds["station_type"] = xr.DataArray(meta_row["station_type"], dims=())

    rivers_raw_datasets.append(ds)


## 3. Add metadata for each data variable 

In [5]:
for ds in rivers_raw_datasets:
    for var in ds.data_vars:
        if var in ["station_id", "station_code", "station_name", "station_type"]:
            continue

        match = pars_metadata_df[pars_metadata_df["parameter_name"] == var]

        if not match.empty:
            row = match.iloc[0]
            param_name = row["parameter_name"]
            ds[var].attrs["parameter_id"] = int(row["parameter_id"])
            ds[var].attrs["units"] = row["unit"]
            ds[var].attrs["parameter_name"] = param_name

            long_name = standard_name_map.get(var, param_name)
            ds[var].attrs["long_name"] = long_name

            # Comment for specific var
            if var == "TOTN_EF_usikker":
                ds[var].attrs["comment"] = (
                    "Total Nitrogen data sourced from a different monitoring program (Eurofins). Used to fill missing values in the primary TOTN dataset."
                )
        else:
            print(f"Warning! No metadata found for variable '{var}'")

# Metadata example
rivers_raw_datasets[2]["TSM"].attrs

{'parameter_id': 1241,
 'units': 'mg/l',
 'parameter_name': 'TSM',
 'long_name': 'Total Suspended Matter'}

## 4. Assign global attributes

In [6]:
for ds in rivers_raw_datasets:
    station_name = ds["station_name"].values.item()
    station_code = ds["station_code"].values.item()

    ds.attrs = dict(
        id = "no.niva:485fc527-3645-4ddc-9128-8f956fab33c2",
        naming_authority="no.niva",
        title=f"Water chemistry measurements at station {station_name}",
        title_no=f"Kjemiske målinger ved stasjon {station_name}",
        summary=f"Long-term water chemistry monitoring at station {station_name} (code: {station_code})",
        summary_no=f"Langsiktige vannkjemiske målinger ved stasjon {station_name} (kode: {station_code})",
        keywords="GCMDSK:EARTH SCIENCE > WATER QUALITY > CHEMISTRY, GCMDLOC:CONTINENT > EUROPE > NORWAY",
        keywords_vocabulary="GCMDSK:GCMD Science Keywords, GCMDLOC:GCMD Locations",
        iso_topic_category="inlandWaters",
        featureType = "timeSeries",
        date_created=datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
        project="AquaINFRA",
        time_coverage_start=np.datetime_as_string(ds.sample_date.min().values, unit="s", timezone="UTC"),
        time_coverage_end=np.datetime_as_string(ds.sample_date.max().values, unit="s", timezone="UTC"),
        geospatial_lat_min=float(ds.latitude.values),
        geospatial_lat_max=float(ds.latitude.values),
        geospatial_lon_min=float(ds.longitude.values),
        geospatial_lon_max=float(ds.longitude.values),
        spatial_representation="point",
        creator_type="institution",
        creator_institution="Norwegian Institute for Water Research (NIVA)",
        institution="NIVA",
        institution_short_name="NIVA",
        creator_email="areti.balkoni@niva.no",
        creator_url="https://www.niva.no/en/employees/areti-balkoni",
        data_owner="Norwegian Institute for Water Research",
        processing_level="Raw - harmonized names",
        Conventions="CF-1.7, ACDD-1.3",
        publisher_name="Norwegian Institute for Water Research",
        publisher_email="miljoinformatikk@niva.no",
        publisher_url="https://niva.no",
        license="http://spdx.org/licenses/CC-BY-4.0(CC-BY-4.0)",
        history="Created on jupyterhub",
    )

# Example dataset
rivers_raw_datasets[2]

C:\Users\ARB\AppData\Local\Temp\ipykernel_24748\725752270.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_created=datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
C:\Users\ARB\AppData\Local\Temp\ipykernel_24748\725752270.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_created=datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
C:\Users\ARB\AppData\Local\Temp\ipykernel_24748\725752270.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_created=datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),


<xarray.Dataset> Size: 65kB
Dimensions:          (sample_date: 425)
Coordinates:
  * sample_date      (sample_date) datetime64[ns] 3kB 1990-01-15 ... 2024-08-...
    latitude         float64 8B 59.1
    longitude        float64 8B 10.07
Data variables: (12/22)
    DOC              (sample_date) float64 3kB nan nan nan nan ... 5.2 3.9 4.7
    Color            (sample_date) float64 3kB nan nan nan ... 41.0 28.0 37.0
    NH4-N            (sample_date) float64 3kB nan nan nan nan ... 13.0 8.0 25.0
    NO3-N            (sample_date) float64 3kB 194.0 410.0 200.0 ... 84.0 130.0
    TRP              (sample_date) float64 3kB 15.5 11.0 6.0 3.5 ... 5.0 4.0 2.0
    POC              (sample_date) float64 3kB nan nan nan ... 497.0 356.0 256.0
    ...               ...
    UV_Abs_254nm     (sample_date) float64 3kB nan nan nan ... 0.1585 0.2058
    UV_Abs_410nm     (sample_date) float64 3kB nan nan nan ... 0.01519 0.02066
    station_id       int64 8B 40355
    station_code     <U4 16B 'NUM1'
    station_name     <U19 76B 'Numedalslågen v/E18'
    station_type     <U3 12B 'Elv'
Attributes: (12/33)
    id:                      no.niva:485fc527-3645-4ddc-9128-8f956fab33c2
    naming_authority:        no.niva
    title:                   Water chemistry measurements at station Numedals...
    title_no:                Kjemiske målinger ved stasjon Numedalslågen v/E18
    summary:                 Long-term water chemistry monitoring at station ...
    summary_no:              Langsiktige vannkjemiske målinger ved stasjon Nu...
    ...                      ...
    Conventions:             CF-1.7, ACDD-1.3
    publisher_name:          Norwegian Institute for Water Research
    publisher_email:         miljoinformatikk@niva.no
    publisher_url:           https://niva.no
    license:                 http://spdx.org/licenses/CC-BY-4.0(CC-BY-4.0)
    history:                 Created on jupyterhub

## 5. Store the datasets 

In [8]:
int_encoding = {"dtype": "int32", "_FillValue": -9999}

for ds in rivers_raw_datasets:
    station_id = ds["station_id"].values.item()
    filename = output_dir / f"raw_riverchem_{station_id}_18-06-2025.nc"

    # Coerce station_id to int32 because some viewers does not support int64
    ds["station_id"] = xr.DataArray(np.int32(station_id), dims=())

    # Set up encoding
    encoding = {
        "sample_date": {
            "dtype": "int32",
            "_FillValue": None,
            "units": "seconds since 1970-01-01 00:00:00",
        },
        "longitude": {"_FillValue": None},
        "latitude": {"_FillValue": None},
        "station_id": int_encoding,  
    }

    ds.to_netcdf(
        path=filename,
        mode="w",
        format="NETCDF4",
        unlimited_dims=["sample_date"],
        encoding=encoding,
    )

    print(f"Saved {filename}")


Saved ..\..\data\river\water_chemistry\raw\raw_riverchem_40352_18-06-2025.nc
Saved ..\..\data\river\water_chemistry\raw\raw_riverchem_40356_18-06-2025.nc
Saved ..\..\data\river\water_chemistry\raw\raw_riverchem_40355_18-06-2025.nc
